### Testing specific issues with the CV's
Some testing... Sara's failed due to the "ć"

In [27]:
import pdfplumber
with pdfplumber.open("C:\Projects\Staffing\data\CV's\Sara_Vera_Marjanovi_.pdf") as pdf:
    print(pdf.pages[0].extract_text()[:-60])
    #for j, page in enumerate(pdf.pages):
    #    print(page.extract_text())

Sara Vera Marjanović
Data Scientist
Data scientist
Sara is a driven data scientist with a particular specialty in Machine Learning, Deep learning, and Natural Language 
Processing. She has end-to-end experience with wrangling a wide variety of different data sources (from free text to 
biometric data) to create either simple (e.g. clustering, regression) and complex models (e.g. Generative Adversarial 
Models) in the production pipeline. Therefore, she is well-versed in ideation, data collection, data wrangling and 
exploration, as well as model development and trouble-shooting and is comfortable with Python (especially the libraries 
Numpy, Pandas, Tensorflow, Pytorch) as well as SQL, R, and Azure DevOps.
Within her model creation and data exploration, Sara takes special care to limit bias within the dataset and model output, 
and to ensure interpretability of the results. She is diligent and conscientious with her data storytelling and data 
visualization, making her a skilled commun

### Extracting text as string from pdf', save to .csv
Here we read the text from all CV's and save the texts in a .csv file. Each line in the .csv contains name of the colleague, the page number of the CV and the text on that specific page. 

In [39]:
import csv
import glob
import regex as re
import pdfplumber
# Get list of CV paths
paths = glob.glob("C:\Projects\Staffing\data\CV's\*.pdf")
#print(paths)

with open("cv_test.csv", "w", newline='') as f:
    writer = csv.writer(f)
    csv_input = ["colleague", "page_number", "page_text"]
    writer.writerows([csv_input])
    for i, path in enumerate(paths):
        print(f"Currently at path: {path[31:-4]}")
        with pdfplumber.open(path) as pdf:
            for j, page in enumerate(pdf.pages):
                # Only special characters are removed.
                csv_input = [path[31:-4],j+1,re.sub('[^A-Za-z0-9 .,]+', '', page.extract_text())]
                #Write page text to pdf
                writer.writerows([csv_input])

Currently at path: Antonio_Arfe
Currently at path: Ashish_Meshram
Currently at path: Ayman_Eleya_Zaki
Currently at path: Aziz_Ari
Currently at path: Branislav_Machava
Currently at path: Daniel_Masson
Currently at path: Daria_Korzel
Currently at path: Devichand_Das
Currently at path: Dinesh_Kumar_Padala
Currently at path: Ditlev_J_rgensen
Currently at path: Drisya_Thumba
Currently at path: Emil_Djursner_Rasmussen
Currently at path: Eugenia_Borgia
Currently at path: Hamza_Minhas
Currently at path: Hatef_Abdollahi
Currently at path: Jacob_Karlstr_m
Currently at path: Jan_Neldeberg
Currently at path: Jayant_Pant
Currently at path: Jes_Melbye_Chergui
Currently at path: Julian_Eikhaug
Currently at path: Kristina_Kj_r_Hedegaard
Currently at path: Ksheeraja_Vinjanampati_
Currently at path: Lee_Ann_Zambrano_Larsen
Currently at path: Line_Bruun
Currently at path: Mohd_Arshad
Currently at path: Mohit_Kumar_Bhati
Currently at path: Mostafa_Ellabaan
Currently at path: Olivia_Essen
Currently at path

### Create TaggedDocument, train Doc2Vec model on data
First we created a TaggedDocument object where we tag each document with the respective name. Each document is a page on the CV.

In [7]:
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
from nltk.tokenize import word_tokenize
import pandas as pd

df = pd.read_csv("cv_test.csv")

tagged_data = []
cnt = 0
for tags in df["colleague"].tolist():
    for doc in df["page_text"].tolist():
        tagged_data.append(TaggedDocument(words=word_tokenize(doc), tags=[cnt, str(tags)]))
    
    cnt += 1

print(tagged_data[0])

#tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(name)]) for name, _d in (df["colleague"], df["page_text"])]
max_epochs = 20
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=5,
                dm =1)

model.build_vocab(tagged_data) # tagged_data

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data, # tagged_data
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v_cvs.model")
print("Model Saved")

TaggedDocument<['Antonio', 'ArfeSenior', 'Big', 'Data', 'EngineerSenior', 'Data', 'Engineer', 'Antonio', 'is', 'an', 'experienced', 'SoftwareData', 'Engineer', 'interested', 'in', 'anything', 'technically', 'challenging', '.', 'He', 'has', 'worked', 'on', 'a', 'wide', 'range', 'of', 'IT', 'projects', 'at', 'Enterprise', 'level', ',', 'gathering', 'a', 'solid', 'experience', 'in', 'productionizing', 'infrastructure', 'applications', 'and', 'data', 'pipelines', 'to', 'support', 'Data', 'Scientists.He', 'worked', 'on', 'two', 'main', 'areas', 'Design', ',', 'implement', 'and', 'maintain', 'infrastructure', 'applications', 'that', 'should', 'run', 'for', 'years', ',', 'be', 'highly', 'available', 'and', 'fault', 'tolerant', 'Build', 'data', 'pipelines', 'and', 'productionize', 'advanced', 'analytics', 'models', 'in', 'constant', 'need', 'of', 'changes', 'and', 'new', 'featuresHIGHLIGHTS', 'Explain', 'any', 'kind', 'of', 'Distributed', 'software', 'system', 'to', 'technical', 'and', 'nontec

### Use trained model to find best match between a subject and a colleague.
Here we use the trained model to match which CV best fits the given text (subject tested on is Data Migration).

In [17]:
model= Doc2Vec.load("d2v_cvs.model")
print("Model loaded!")


# Test random text on data migration
doc = """Data Migration
Data-Migration-Icon-603x603What is Data Migration?
Data Migration is the process of selecting and moving data from one location to another – this may involve moving data across different storage vendors, and across different formats.

Data migrations are often done in the context of retiring a system and moving to a new system, or in the context of a cloud migration, or in the context of a modernization or upgrade strategy.

Data migrations can be laborious, error prone, manual, and time consuming. Migrating data may involve finding and moving billions of files, which can succumb to storage and network slowdowns or outages. Also, different file systems do not often preserve metadata in exactly the same way, so migrating data without loss of fidelity and integrity can be a challenge.

Network Attached Storage (NAS) migration is the process of migrating from one NAS storage environment to another. This may involve migrations within a vendor’s ecosystem such as NetApp data migration to NetApp or across vendors such as NetApp data migration to Isilon or EMC to NetApp or EMC to Pure FlashBlade. A high-fidelity NAS migration solution should preserve not only the file itself but all of its associated metadata and access controls.

Network Attached Storage (NAS) to Cloud data migration is the process of moving data from an on-premises data center to a cloud. It requires data to be moved from a file format (NFS or SMB) to an Object/Cloud format such as S3. A high-fidelity NAS-to-Cloud migration solution preserves all the file metadata including access control and privileges in the cloud. This enables data to be used either as objects or as files in the cloud.

Storage migration is a general-purpose term that applies to moving data across storage arrays.

Data migrations typically involve four phases:

Planning – Deciding what data should be migrated. Planning may often involve analyzing various sources to find the right data sets. For example, several customers today are interested in upgrading some data to Flash – finding hot, active data to migrate to Flash can be a useful planning exercise.
Initial Migration – Do a first migration of all the data. This should involve migrating the files, the directories and the shares.
Iterative Migrations – Look for any changes that may have occurred during the initial migration and copy those over.
Final Cutoff – A final cutoff involves deleting data at the original storage and managing the mounts, etc., so data can be accessed from the new location going forward.
Resilient data migration refers to an approach that automatically adjusts for failures and slowdowns and retries as needed. It also checks the integrity of the data at the destination to ensure full fidelity.

Types of data migration
When it comes to file data, there are NAS Migrations and Cloud Migrations. There are also NAS migrations to the cloud. Data migrations are often seen as a dreaded and laborious part of the storage management lifecycle. Free tools are often considered first but they can introduce risk, time and cost overruns and they are typically labor intensive and error-prone. On the other hand, traditional migration tools have complex legacy architectures and are expensive point products that do not provide ongoing value – resulting in sunk costs.

Look for simple, fast, reliable data migration tools are not one-and-done point tools. The right data migration solution should be able to handle other data management use cases, including data tiering and data replication.

How to plan a Smart NAS or Cloud data migration?
The typical steps for any data migration project are:

Analytics: Before you start an unstructured data migration project, it’s important to have visibility into:

 How fast is your data growing?
 How much data is hot vs. cold cold?
 Who is using your data?
Savings: Estimate how much you’ll save by moving to the new NAS or cloud infrastructure. This information will guide which NAS or cloud storage mix is best for your data.

Offload heavy lifting: Your data migration solution should be able to manage multiple iterations of the migration and handle problems by automatically retrying in a
slowdown or a network or storage failure.

Preserve data integrity: Your data migration solution should provide MD5 checksum on every file and assure all metadata and access controls migrate to the new environment.

Avoid sunk costs: File data migrations are a lot of heaving lifting. Your data migration solution should include automatic parallelization at every level for elastic
scaling and the ability to migrate petabytes of data seamlessly and reliably.

Reduce downtime: It is recommended that your data migration solution runs multiple iterations for more efficient cutovers.

Komprise and data migration
Komprise Elastic Data Migration is included in the Komprise Intelligent Data Management platform or is available standalone. Designed for cloud migrations and NAS migrations, with Komprise Elastic Data Migration you can run, monitor, and manage hundreds of data migrations faster than ever at a fraction of the cost. Learn more about Komprise Smart Data Migrations.

Data migration and the cloud
As unstructured data continues to grow exponentially, organizations struggle to control costs for file data storage. Many are turning to the cloud to scale and manage spend. However, choosing the right files to move can be challenging as there can easily be billions of files. Many enterprises have over 1 PB of data, which represents roughly 3 billion files. This unstructured data is growing exponentially and resides in multi-vendor storage silos for access by various applications and departments.
For these reasons, organizations often lack visibility into file data and are making decisions in the dark. To be agile and competitive, IT teams must evolve storage management to become a holistic data management strategy. The right approach to data migration and the cloud for file and object data is to use analytics in cloud data management:

Understand your data patterns
Plan using a cost model
Use data to drive stakeholder buy-in
Eliminate user disruption
Create a systematic plan for ongoing data management
Read the eBook: 5 Ways to Use Analytics for Cloud Data Migrations

Top data migration challenges
Businesses today are looking at modernizing storage and moving to a multi-cloud strategy. As they evolve to faster, flash-based Network Attached Storage (NAS) and the cloud, migrating data into these environments can be challenging. The goal is to migrate large production data sets quickly, without errors, and without disruption to user productivity.

The top cloud data migration challenges are:

How do you manage cloud data migrations without downtime?
How can you automate cloud data migrations to eliminate manual effort?
How can you ensure all the permissions, ACLs, metadata are copied correctly during a cloud data migration so you can access the data in the cloud as files?
You can overcome these challenges with some planning and automation that preserves file-based access both from on-premises and the cloud.

Data migration tools
Free Tools: Require a lot of babysitting and are not reliable for migrating large volumes of data.
Point Data Migration Solutions: Have complex legacy architectures and create sunk costs.
Komprise Elastic Data Migration: Makes cloud data migrations simple, fast, reliable and eliminates sunk costs since you continue to use Komprise after the migration. Komprise is the only solution that gives you the option to cut 70%+ cloud storage costs by placing cold data in Object classes while maintaining file metadata so it can be promoted in the cloud as files when needed."""

# Get vector of test text
test_vector = model.infer_vector(word_tokenize(doc))

# Get all CV's that match
best_matches = model.dv.similar_by_vector(test_vector)

#See all matches
print(best_matches)
print(best_matches[0][0])
# Print the text of the best match
# To analyse if there match is decent or not.
print(tagged_data[0])

Model loaded!
[('Prashanthy_Ramanan', 0.685684859752655), ('Hamza_Minhas', 0.6856452226638794), ('Ditlev_J_rgensen', 0.6827963590621948), ('Ashish_Meshram', 0.6826142072677612), ('Line_Bruun', 0.6804088950157166), ('Saket_Kumar', 0.6799563765525818), ('Kristina_Kj_r_Hedegaard', 0.6793414354324341), ('Peter_Damm', 0.6740396618843079), ('Daria_Korzel', 0.6727231740951538), ('Mohd_Arshad', 0.6713230609893799)]
Prashanthy_Ramanan
TaggedDocument<['Antonio', 'ArfeSenior', 'Big', 'Data', 'EngineerSenior', 'Data', 'Engineer', 'Antonio', 'is', 'an', 'experienced', 'SoftwareData', 'Engineer', 'interested', 'in', 'anything', 'technically', 'challenging', '.', 'He', 'has', 'worked', 'on', 'a', 'wide', 'range', 'of', 'IT', 'projects', 'at', 'Enterprise', 'level', ',', 'gathering', 'a', 'solid', 'experience', 'in', 'productionizing', 'infrastructure', 'applications', 'and', 'data', 'pipelines', 'to', 'support', 'Data', 'Scientists.He', 'worked', 'on', 'two', 'main', 'areas', 'Design', ',', 'implemen